In [1]:
from bitarray import bitarray
import struct
import sys

In [2]:
sys.path.insert(1, '/home/chholak/Git/Cryptography/Chapter4')
from des import DES

In [3]:
def string2bits(string):
    lst = []
    for c in string:
        bits = bin(ord(c))[2:]
        bits = '00000000'[len(bits):] + bits
        lst.extend([int(b) for b in bits])
    return bitarray(lst)

bitarray2int = lambda bit_array: struct.unpack(">Q", bit_array)[0]

def bitarray2str(bit_array): #Recreate the string from the bit array
    bits = [int(i) for i in bit_array.tolist()]
    chars = []
    for b in range(len(bits) // 8):
        byte = bits[b * 8 : (b + 1) * 8]
        chars.append(chr(int(''.join([str(bit) for bit in byte]), 2)))
    return ''.join(chars)

In [27]:
pwd = "baA"

# Convert to uppercase
pwd = pwd.upper()

# Add null padding to fill 14 bytes
null_bytes = 14 - len(pwd)
print("Null bytes:", null_bytes)
bit_array = string2bits(pwd)
for i in range(null_bytes * 8):
    bit_array.append(0)
key = bitarray2str(bit_array)
print("Key: %r" % key)

# Add a zero after every 7 bits


Null bytes: 11
Key: 'BAA\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


In [11]:
null_bits = [0] * 64
bit_array = bitarray(null_bits)
key = bitarray2str(bit_array)
print("Key: %r" % key)

Key: '\x00\x00\x00\x00\x00\x00\x00\x00'


In [5]:
text = "KGS!@#$%"

In [6]:
c = DES.encrypt(text, key, padding=False)
print("Cipher: %r" % c)

Cipher: 'ªÓ´5µ\x14\x04î'


In [7]:
DES.decrypt(c, key, padding=False)

'KGS!@#$%'

In [8]:
hex(bitarray2int(string2bits(c)))

'0xaad3b435b51404ee'